In [1]:
# import the necessary libraries
import numpy as np
import pandas as pd

# import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


In [2]:
# Display all the columns in the dataset
pd.set_option('max_columns', None)

# Display at least 100 rows in the dataset
pd.set_option('max_rows', 100)

import re

In [3]:
# load the data
players_df = pd.read_csv('CompleteDataset.csv', index_col=0)
players_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (23,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Acceleration,Aggression,Agility,Balance,Ball control,Composure,Crossing,Curve,Dribbling,Finishing,Free kick accuracy,GK diving,GK handling,GK kicking,GK positioning,GK reflexes,Heading accuracy,Interceptions,Jumping,Long passing,Long shots,Marking,Penalties,Positioning,Reactions,Short passing,Shot power,Sliding tackle,Sprint speed,Stamina,Standing tackle,Strength,Vision,Volleys,CAM,CB,CDM,CF,CM,ID,LAM,LB,LCB,LCM,LDM,LF,LM,LS,LW,LWB,Preferred Positions,RAM,RB,RCB,RCM,RDM,RF,RM,RS,RW,RWB,ST
0,Cristiano Ronaldo,32,https://cdn.sofifa.org/48/18/players/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Real Madrid CF,https://cdn.sofifa.org/24/18/teams/243.png,€95.5M,€565K,2228,89,63,89,63,93,95,85,81,91,94,76,7,11,15,14,11,88,29,95,77,92,22,85,95,96,83,94,23,91,92,31,80,85,88,89.0,53.0,62.0,91.0,82.0,20801,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,ST LW,89.0,61.0,53.0,82.0,62.0,91.0,89.0,92.0,91.0,66.0,92.0
1,L. Messi,30,https://cdn.sofifa.org/48/18/players/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,93,93,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,€105M,€565K,2154,92,48,90,95,95,96,77,89,97,95,90,6,11,15,14,8,71,22,68,87,88,13,74,93,95,88,85,26,87,73,28,59,90,85,92.0,45.0,59.0,92.0,84.0,158023,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,RW,92.0,57.0,45.0,84.0,59.0,92.0,90.0,88.0,91.0,62.0,88.0
2,Neymar,25,https://cdn.sofifa.org/48/18/players/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,94,Paris Saint-Germain,https://cdn.sofifa.org/24/18/teams/73.png,€123M,€280K,2100,94,56,96,82,95,92,75,81,96,89,84,9,9,15,15,11,62,36,61,75,77,21,81,90,88,81,80,33,90,78,24,53,80,83,88.0,46.0,59.0,88.0,79.0,190871,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,LW,88.0,59.0,46.0,79.0,59.0,88.0,87.0,84.0,89.0,64.0,84.0
3,L. Suárez,30,https://cdn.sofifa.org/48/18/players/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,92,92,FC Barcelona,https://cdn.sofifa.org/24/18/teams/241.png,€97M,€510K,2291,88,78,86,60,91,83,77,86,86,94,84,27,25,31,33,37,77,41,69,64,86,30,85,92,93,83,87,38,77,89,45,80,84,88,87.0,58.0,65.0,88.0,80.0,176580,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,ST,87.0,64.0,58.0,80.0,65.0,88.0,85.0,88.0,87.0,68.0,88.0
4,M. Neuer,31,https://cdn.sofifa.org/48/18/players/167495.png,Germany,https://cdn.sofifa.org/flags/21.png,92,92,FC Bayern Munich,https://cdn.sofifa.org/24/18/teams/21.png,€61M,€230K,1493,58,29,52,35,48,70,15,14,30,13,11,91,90,95,91,89,25,30,78,59,16,10,47,12,85,55,25,11,61,44,10,83,70,11,NaN,NaN,NaN,NaN,NaN,167495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Information on the data
players_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17981 entries, 0 to 17980
Data columns (total 74 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Name                 17981 non-null  object 
 1   Age                  17981 non-null  int64  
 2   Photo                17981 non-null  object 
 3   Nationality          17981 non-null  object 
 4   Flag                 17981 non-null  object 
 5   Overall              17981 non-null  int64  
 6   Potential            17981 non-null  int64  
 7   Club                 17733 non-null  object 
 8   Club Logo            17981 non-null  object 
 9   Value                17981 non-null  object 
 10  Wage                 17981 non-null  object 
 11  Special              17981 non-null  int64  
 12  Acceleration         17981 non-null  object 
 13  Aggression           17981 non-null  object 
 14  Agility              17981 non-null  object 
 15  Balance              17981 non-null 

In [5]:
players_df['Preferred Positions'].unique()

array(['ST LW ', 'RW ', 'LW ', 'ST ', 'GK ', 'CDM CM ', 'CB ',
       'RM CM CAM ', 'RM LW ST LM ', 'ST CAM ', 'LW RW ST ',
       'CDM CAM CM ', 'RW CAM ', 'RW RM ', 'CM CDM ', 'LM RM CAM ', 'LB ',
       'CM ', 'LM CM CAM ', 'LM CM ', 'LM RM CAM CM ', 'CM CB LB ',
       'LM LB ', 'ST CAM RW CF ', 'CAM LW ', 'LW ST LM ', 'CAM CM ',
       'CDM CB ', 'ST LW CF ', 'CAM CDM CM ', 'LW LM ', 'RM LM ', 'CDM ',
       'LM RW RM ', 'LB RB CB ', 'LW RW ', 'LW LM RM CAM ', 'RM ',
       'LM CF CAM ', 'RB ', 'LM CAM LW ', 'LM RM CM ', 'CAM LM ',
       'CAM RM RW ', 'RM CAM ', 'CAM LW RW ', 'LM ', 'LM CAM ',
       'CAM CF ST ', 'CM CB CDM ', 'RW ST ', 'RW LW CM ', 'CF CAM ',
       'LW CM ', 'RWB RB ', 'CAM ', 'ST CF ', 'LM ST ', 'CM CAM ',
       'CF ST ', 'CB RB ', 'CF CM CAM ', 'LM CAM RW LW ', 'LM RM ',
       'LM RW LW ', 'LM CF RM ', 'CAM LW CM ', 'ST LM ', 'LM ST RM ',
       'RM RW ', 'RM CM ', 'RB CB ', 'CM RW RB ', 'LM RM RW ', 'RW LW ',
       'CM CDM CB ', 'LB CB ', 'ST RM LM ', 'C

### Questions
**1)** Who are the best players based on their positions? i.e ST, LW, RW, CAM, CM, CDM, LB, RB, LWB, RWB, CB, GK

**2)** Who are the top earners?

**3)** Who are the least earners?

**4)** Which players have the most value?

**5)** Which players have the least value?

**6)** Which nationality is most represented in the data set?

**7)** Which nationality is least represented in the data set?

**8)** Which players have the most potential?

**9)** Which players are more versatile in terms of 'Preferred Positions'?

**10)** Best penalty, freekick takers?

**11)** Which players have the most aggression? Attackers or Midfielders??

**12)** Which players have the most stamina, ball control? Defenders or Midfielders??

**13)** Which players have the most positioning, jumping? Defenders or Attackers??

In [8]:
# A copy of the data
df = players_df.copy()

#### Data Wrangling
* The data contains a lot of rows and columns that need to be cleaned up.

In [9]:
def split_positions(position):
    """
    This returns a list that's stripped of white spaces.
    eg 'LW LS ' ==> 'LW,LS' 
    """
    value = ''
    lst = position.split()
    lgth = len(lst)
    
    for idx, val in enumerate(lst):
        if (idx + 1) != lgth:  # if it's not the last element
            value += f"{val},"
        else:
            value += f"{val}"
            
    return value

In [14]:
# Remove the whitespaces in 'Preferred Positions'
df['Preferred Positions'] = df['Preferred Positions'].apply(split_positions)

df.sample(3, random_state=6)[['Name', 'Preferred Positions']]

,Name,Preferred Positions
280,C. Vela,"ST,RW"
569,J. Stones,CB
8731,V. Vermeij,ST


**Best players by position**

In [15]:
def positions(position):
    """
    This function returns the position of a player. eg 'RM,CDM,CM' ==> 'Attacker',
    'RM,CM,RB' ==> 'Midfielder' 
    """
    lst = position.split(',')
    value = ''
    for val in lst:
        if val in ["ST", "CF", "LW", "RW", "LS", "RS", 'CAM']:
            value += "Attacker "
            
        elif val in ['CDM', 'CM', 'CAM', 'LM', 'RM']:
            value += "Midfielder "
         
        elif val in ['RB', 'LB', 'RWB', 'LWB', 'CB']:
            value += "Defender "
        
        else:
            value += "Goalkeeper"
    value = value.split()[0]
    
    return value.strip()

In [16]:
# Group the players into positions which could be:  "Attacker", "Midfielder", "Defender", or "Goalkeeper" 
df['Position'] = df['Preferred Positions'].apply(positions)

<br>

**Best Attackers:**

In [21]:
# Best Attackers
df.loc[df['Position'] == 'Attacker', 
       ['Name', 'Club', 'Overall', 'Value', 'Wage', 'Finishing', 'Dribbling','Ball control']][:5]

,Name,Club,Overall,Value,Wage,Finishing,Dribbling,Ball control
0,Cristiano Ronaldo,Real Madrid CF,94,€95.5M,€565K,94,91,93
1,L. Messi,FC Barcelona,93,€105M,€565K,95,97,95
2,Neymar,Paris Saint-Germain,92,€123M,€280K,89,96,95
3,L. Suárez,FC Barcelona,92,€97M,€510K,94,86,91
5,R. Lewandowski,FC Bayern Munich,91,€92M,€355K,91,85,89


<br>

**Best Midfielders:**

In [22]:
# Best Midfielders
df.loc[df['Position'] == 'Midfielder', 
       ['Name', 'Club', 'Overall', 'Value', 'Wage', 'Vision', 'Short passing', 'Agility', 'Standing tackle']][:5]

,Name,Club,Overall,Value,Wage,Vision,Short passing,Agility,Standing tackle
8,T. Kroos,Real Madrid CF,90,€79M,€340K,88,90,71,82
11,K. De Bruyne,Manchester City,89,€83M,€285K,90,90,80,51
13,A. Sánchez,Arsenal,89,€67.5M,€265K,83,81,90,39
14,L. Modrić,Real Madrid CF,89,€57M,€340K,90,92,93,80
22,Thiago,FC Bayern Munich,88,€70.5M,€225K,86,91,90,64


<br>

**Best Defenders:**

In [23]:
# Best Defenders
df.loc[df['Position'] == 'Defender', 
       ['Name', 'Club', 'Overall', 'Value', 'Wage', 'Marking', 'Standing tackle', 'Jumping',]][:5]

,Name,Club,Overall,Value,Wage,Marking,Standing tackle,Jumping
10,Sergio Ramos,Real Madrid CF,90,€52M,€310K,86,89,93
17,G. Chiellini,Juventus,89,€38M,€225K,92,92,89
24,L. Bonucci,Milan,88,€44M,€210K,84,88,85
25,J. Boateng,FC Bayern Munich,88,€48M,€215K,88,91,75
26,D. Godín,Atlético Madrid,88,€40M,€125K,87,86,89


<br>

**Best Goalkeepers:**

In [25]:
# Best Goalkeepers
df.loc[df['Position'] == 'Goalkeeper', 
       ['Name', 'Club', 'Overall', 'Value', 'Wage', 'GK handling', 'GK positioning','GK reflexes']][:5]

,Name,Club,Overall,Value,Wage,GK handling,GK positioning,GK reflexes
4,M. Neuer,FC Bayern Munich,92,€61M,€230K,90,91,89
6,De Gea,Manchester United,90,€64.5M,€215K,85,86,90
12,T. Courtois,Chelsea,89,€59M,€190K,91,86,88
18,G. Buffon,Juventus,89,€4.5M,€110K,88,90,84
20,J. Oblak,Atlético Madrid,88,€57M,€82K,90,87,84


In [26]:
# A copy of the data
df_1 = df.copy()

In [27]:
attributes = ['Acceleration', 'Aggression', 'Agility', 'Balance', 'Ball control', 'Composure', 'Crossing', 'Curve',  
    'Dribbling', 'Finishing', 'Free kick accuracy','GK diving', 'GK handling', 'GK kicking', 
    'GK positioning', 'GK reflexes', 'Heading accuracy', 'Interceptions', 'Jumping','Long passing', 
    'Long shots', 'Marking', 'Penalties', 'Positioning','Reactions', 'Short passing', 
    'Shot power', 'Sliding tackle','Sprint speed', 'Stamina', 'Standing tackle', 'Strength', 'Vision','Volleys']

In [29]:
# for col in attributes:
#     print(f"'{col}'==> \n{df_1[col].unique()}\n")
#     print("===" * 30)

In [30]:
# Remove '+' and '-' from each row containing it. 
for col in attributes:
    df_1[col] = np.where(df_1[col].str.contains('\+|-'), 
                         df_1[col].str[:2], 
                         df_1[col])

# # Verify that the columns have been cleaned
# for col in attributes:
#     print(f"'{col}'==> \n{df_1[col].unique()}\n")
#     print("===" * 30)

In [32]:
# Convert the columns to float
for col in attributes:
    df_1[col] = df_1[col].astype('float') 

# # Verify!    
# df_1.info()

In [33]:
# A copy of the data
df_2 = df_1.copy()

# A view of the data
df_2.loc[3:5, ['Name', 'Wage', 'Value', 'Position']]

,Name,Wage,Value,Position
3,L. Suárez,€510K,€97M,Attacker
4,M. Neuer,€230K,€61M,Goalkeeper
5,R. Lewandowski,€355K,€92M,Attacker


In [35]:
# # Determine the unoque values in ['Value', 'Wage']
# for col in ['Value', 'Wage']:
#     print(f"{col}==> \n{df_2[col].unique()}")
#     print("===" *28)

In [36]:
# Replace '€0' with '€0K'. This is useful for the string manipulation.
df_2['Value'] = np.where(df_2['Value'] == '€0', '€0K', df_2['Value'])
df_2['Wage'] = np.where(df_2['Wage'] == '€0', '€0K', df_2['Wage'])

In [37]:
df_2['Currency'] = df_2['Value'].str[0]
df_2['Unit'] = df_2['Value'].str[-1]

# Verify that the 'Currency' and 'Unit' were properly extracted
# df_2.loc[:2, ['Name', 'Wage', 'Value', 'Currency', 'Unit', 'Position']]

In [39]:
cols = df_2.columns.tolist()

# rearrange the columns
cols = cols[:9] + cols[-2:] + cols[9:-2]
df_2 = df_2[cols]

# df_2.head(2)

In [40]:
# Extract the Value
df_2['Value'] = df_2['Value'].str[1:-1]

# Extract the Wage
df_2['Wage'] = df_2['Wage'].str[1:-1]

# Rename column from 'Wage' to 'Wage (K)'
df_2 = df_2.rename(columns={'Wage': 'Wage (K)'})

In [41]:
# Verify that the operations in the cell above was properly performed.
# df_2.sample(2, random_state=1)

In [42]:
# # Check for unique values in 'Value', and 'Wage (K)'
# for col in ['Value', 'Wage (K)']:
#     print(f"{col}==> \n {df_2[col].unique()}")
#     print("===" *28)

In [43]:
# Convert to float
for col in ['Value', 'Wage (K)']:
    df_2[col] = df_2[col].astype('float')

# # Verify    
# df_2[['Value', 'Wage (K)']].info()

In [44]:
# Make a copy of the data
df_3 = df_2.copy()

# # Save the cleaned data
# df_3.to_csv('CompleteDataset_cleaned.csv', index=False)